In [4]:
import matplotlib.pyplot as plt 
import numpy as np


In [5]:
def parse_interactome(file_path):
    interactome_data = {}
    with open(file_path, 'r') as file:
        next(file)  # Skip the header line
        for line in file:
            tail, head, edge_weight, _ = line.strip().split('\t')
            edge_weight = float(edge_weight)
            if tail not in interactome_data:
                interactome_data[tail] = []
            interactome_data[tail].append((head, edge_weight))
    return interactome_data

In [6]:
def list_connected_proteins(file_path, protein):
    interactome_data = parse_interactome(file_path)
    connected_proteins = interactome_data.get(protein, [])
    with open(f"{protein}_connected_proteins.txt", 'w') as output_file:
        # degree of protein: number of connections
        output_file.write(f"Degree of {protein}: {len(connected_proteins)}\n")
        for connected_protein, weight in connected_proteins:
            output_file.write(f"{connected_protein}\t{weight}\n")

In [8]:
# Main code
file_path = "PathLinker_2018_human-ppi-weighted-cap0_75.txt"
protein_of_interest = "Q8TBF4"
interactome_data = parse_interactome(file_path) 
list_connected_proteins (file_path, protein_of_interest)